### Imports

In [645]:
import pandas as pd
import warnings
from sklearn.preprocessing import MinMaxScaler

warnings.filterwarnings('ignore')

from darts import TimeSeries
from darts.models import RNNModel
from darts.dataprocessing.transformers import Scaler
from darts.metrics import mape

import numpy as np
import talib
from darts.models import KalmanFilter
from darts.models import KalmanForecaster

# from gluonts.dataset.common import ListDataset
# from gluonts.model.deepar import DeepAREstimator
# from gluonts.model.simple_feedforward import SimpleFeedForwardEstimator
# from gluonts.mx.trainer import Trainer
# from gluonts.evaluation import Evaluator

In [646]:
# Evito que ciertas columnas se transformen a notacion cientifica en las predicciones
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)

In [647]:
columns = [
    # 'Open_time',
    'Open',
    'High',
    'Low',
    'Close',
    # 'Number of trades',
    # 'EMA_20',
    # 'Upper_Band',
    # 'Middle_Band',
    # 'RSI',
    # 'Buy_1000x_high',
    # 'sell_1000x_high'
]

columns_to_model = [
    # 'Open_time',
    'Open',
    'High',
    'Low',
    'SMA_20',
    'EMA_20',
    'Upper_Band',
    'Middle_Band',
    'Lower_Band',
    'RSI',
    'MACD',
    'Signal',
    'ADX',
    'SlowK',
    'SlowD',
    'CCI',
    'ATR',
]

### Model

In [648]:
complete_dataset = pd.read_csv('/Users/mmarchetta/Desktop/Tesis-2024/data-visualization/final_dataset.csv') 
classifier_dataset = complete_dataset[columns]

display(classifier_dataset)

,Open,High,Low,Close
0,28.84,30.26,27.50,27.71
1,27.72,28.38,26.14,26.31
2,26.31,28.59,26.11,27.28
3,27.28,28.99,27.13,28.62
4,28.61,32.33,28.50,31.94
...,...,...,...,...
951,7.12,7.40,7.01,7.30
952,7.30,7.47,7.06,7.12
953,7.12,7.29,6.98,6.99
954,6.99,7.19,6.89,6.98


In [649]:

features = classifier_dataset[columns]
features['timestamp'] = pd.date_range(start='2021-09-27', periods=len(features), freq='D')
features.set_index('timestamp', inplace=True)
display(features)

series = TimeSeries.from_dataframe(features)

,Open,High,Low,Close
timestamp,,,,
2021-09-27,28.84,30.26,27.50,27.71
2021-09-28,27.72,28.38,26.14,26.31
2021-09-29,26.31,28.59,26.11,27.28
2021-09-30,27.28,28.99,27.13,28.62
2021-10-01,28.61,32.33,28.50,31.94
...,...,...,...,...
2024-05-05,7.12,7.40,7.01,7.30
2024-05-06,7.30,7.47,7.06,7.12
2024-05-07,7.12,7.29,6.98,6.99


In [650]:
train, val = series[:-10], series[-10:]

In [651]:
display(series)

<TimeSeries (DataArray) (timestamp: 956, component: 4, sample: 1)>
array([[[28.84 ],
        [30.26 ],
        [27.5  ],
        [27.71 ]],

       [[27.72 ],
        [28.38 ],
        [26.14 ],
        [26.31 ]],

       [[26.31 ],
        [28.59 ],
        [26.11 ],
        [27.28 ]],

       ...,

       [[ 7.119],
        [ 7.295],
        [ 6.979],
        [ 6.993]],

       [[ 6.992],
        [ 7.189],
        [ 6.893],
        [ 6.983]],

       [[ 6.982],
        [ 7.089],
        [ 6.782],
        [ 7.05 ]]])
Coordinates:
  * timestamp  (timestamp) datetime64[ns] 2021-09-27 2021-09-28 ... 2024-05-09
  * component  (component) object 'Open' 'High' 'Low' 'Close'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [652]:
#Kalman + RNNModel
# kf = KalmanFilter(dim_x=4)
# kf.fit(train)

# train_filtered = kf.filter(train)
# val_filtered = kf.filter(val)

# # Configura y entrena el modelo
# model = RNNModel(model='LSTM',
#                  input_chunk_length=20,
#                  output_chunk_length=10,
#                  batch_size=32,
#                  n_epochs=200,
#                  hidden_dim=256,
#                  n_rnn_layers=2,
#                  dropout=0.2,
#                  optimizer_kwargs={'lr': 0.001},
#                  save_checkpoints=True,
#                  model_name="LSTM_RNN",
#                  force_reset=True
# ) 

# model.fit(train_filtered)



# from darts.models import NBEATSModel

# model = NBEATSModel(
#     input_chunk_length=30,  # longitud del pasado que usaremos para predecir
#     output_chunk_length=10,  # longitud de la predicción
#     n_epochs=100,            # número de épocas de entrenamiento
#     random_state=42
# )

# model.fit(train)


# KalmanForecaster
model = KalmanForecaster(dim_x=4)
model.fit(train)

KalmanForecaster(dim_x=4, kf=None, add_encoders=None)

In [653]:
#Kalman + RNNModel
# best_model = model.load_from_checkpoint(model_name="LSTM_RNN", best=False)
# pred_series = best_model.predict(n=len(val))

#NBEATS
# pred_series = model.predict(n=len(val))

# KalmanForecaster
pred_series = model.predict(n=len(val))


# Evalúa las predicciones
print(f"MAPE: {mape(val, pred_series)}")

MAPE: 11.000316946223691


In [654]:
# pred_series = best_model.predict(n=len(val))
pred_series = model.predict(n=len(val))
pred_df = pred_series.pd_dataframe()

display(pred_df)

component,Open,High,Low,Close
timestamp,,,,
2024-04-30,6.53,6.75,6.20,6.47
2024-05-01,6.43,6.65,6.10,6.36
2024-05-02,6.35,6.57,6.02,6.29
2024-05-03,6.29,6.52,5.98,6.25
2024-05-04,6.26,6.49,5.96,6.24
2024-05-05,6.24,6.47,5.95,6.22
2024-05-06,6.21,6.44,5.92,6.19
2024-05-07,6.17,6.40,5.88,6.14
2024-05-08,6.12,6.34,5.82,6.08


In [655]:
def calcular_indicadores_tecnicos(historico_precio):
    close_prices = np.array(historico_precio['Close'], dtype=np.double)
    high_prices = np.array(historico_precio['High'], dtype=np.double)
    low_prices = np.array(historico_precio['Low'], dtype=np.double)
    
    # Media Móvil Simple (SMA)
    sma_20 = talib.SMA(close_prices, timeperiod=20)
    
    # Media Móvil Exponencial (EMA)
    ema_20 = talib.EMA(close_prices, timeperiod=20)
    
    # Bandas de Bollinger
    upper_band, middle_band, lower_band = talib.BBANDS(close_prices, timeperiod=20, nbdevup=2, nbdevdn=2)
    
    # Índice de Fuerza Relativa (RSI)
    rsi = talib.RSI(close_prices, timeperiod=14)
    
    # MACD (Convergencia/Divergencia de Medias Móviles)
    macd, signal, _ = talib.MACD(close_prices, fastperiod=12, slowperiod=26, signalperiod=9)
    
    # ADX (Índice Direccional Promedio)
    adx = talib.ADX(high_prices, low_prices, close_prices, timeperiod=14)
    
    # Estocástico
    slowk, slowd = talib.STOCH(high_prices, low_prices, close_prices, fastk_period=14, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
    
    # Índice de Canal de Materias Primas 
    cci = talib.CCI(high_prices, low_prices, close_prices, timeperiod=14)
    
    # ATR (Average True Range)
    atr = talib.ATR(high_prices, low_prices, close_prices, timeperiod=14)
    
    indicadores = pd.DataFrame({
        'SMA_20': sma_20,
        'EMA_20': ema_20,
        'Upper_Band': upper_band,
        'Middle_Band': middle_band,
        'Lower_Band': lower_band,
        'RSI': rsi,
        'MACD': macd,
        'Signal': signal,
        'ADX': adx,
        'SlowK': slowk,
        'SlowD': slowd,
        'CCI': cci,
        'ATR': atr
    }, index=historico_precio.index)
    
    return indicadores

In [656]:
classifier_dataset_temp = classifier_dataset
classifier_dataset_temp = pd.concat([classifier_dataset_temp, pd.DataFrame(pred_df[-10:])], axis=0)


In [657]:
indicadores = calcular_indicadores_tecnicos(classifier_dataset_temp)
indicadores = indicadores.tail(10)
ultimos_10_registros = classifier_dataset_temp.tail(10)

In [658]:
display(indicadores)

,SMA_20,EMA_20,Upper_Band,Middle_Band,Lower_Band,RSI,MACD,Signal,ADX,SlowK,SlowD,CCI,ATR
2024-04-30 00:00:00,6.97,7.05,7.52,6.97,6.42,37.52,-0.21,-0.25,27.59,55.35,63.40,-112.96,0.49
2024-05-01 00:00:00,6.93,6.98,7.54,6.93,6.33,36.20,-0.24,-0.25,28.34,40.92,54.48,-119.39,0.50
2024-05-02 00:00:00,6.87,6.92,7.49,6.87,6.26,35.24,-0.27,-0.25,29.18,23.57,39.94,-113.54,0.50
2024-05-03 00:00:00,6.82,6.85,7.47,6.82,6.18,34.76,-0.29,-0.26,30.03,19.78,28.09,-106.71,0.50
2024-05-04 00:00:00,6.79,6.80,7.48,6.79,6.10,34.57,-0.30,-0.27,30.85,18.37,20.57,-104.56,0.50
2024-05-05 00:00:00,6.76,6.74,7.49,6.76,6.02,34.33,-0.31,-0.28,31.64,18.20,18.78,-92.67,0.51
2024-05-06 00:00:00,6.73,6.69,7.50,6.73,5.95,33.81,-0.32,-0.29,32.42,17.82,18.13,-82.97,0.51
2024-05-07 00:00:00,6.69,6.64,7.51,6.69,5.88,33.00,-0.33,-0.29,33.24,17.18,17.73,-79.35,0.51
2024-05-08 00:00:00,6.66,6.58,7.52,6.66,5.81,32.09,-0.33,-0.30,34.11,16.45,17.15,-79.57,0.51
2024-05-09 00:00:00,6.63,6.53,7.53,6.63,5.74,31.30,-0.33,-0.31,35.03,15.83,16.49,-83.31,0.51


In [659]:
indicadores.reset_index(drop=True, inplace=True)
ultimos_10_registros.reset_index(drop=True, inplace=True)

# Fusionar los DataFrames indicadores y ultimos_10_registros basándose en los índices
result = pd.concat([indicadores, ultimos_10_registros], axis=1)
display(result)

,SMA_20,EMA_20,Upper_Band,Middle_Band,Lower_Band,RSI,MACD,Signal,ADX,SlowK,SlowD,CCI,ATR,Open,High,Low,Close
0,6.97,7.05,7.52,6.97,6.42,37.52,-0.21,-0.25,27.59,55.35,63.40,-112.96,0.49,6.53,6.75,6.20,6.47
1,6.93,6.98,7.54,6.93,6.33,36.20,-0.24,-0.25,28.34,40.92,54.48,-119.39,0.50,6.43,6.65,6.10,6.36
2,6.87,6.92,7.49,6.87,6.26,35.24,-0.27,-0.25,29.18,23.57,39.94,-113.54,0.50,6.35,6.57,6.02,6.29
3,6.82,6.85,7.47,6.82,6.18,34.76,-0.29,-0.26,30.03,19.78,28.09,-106.71,0.50,6.29,6.52,5.98,6.25
4,6.79,6.80,7.48,6.79,6.10,34.57,-0.30,-0.27,30.85,18.37,20.57,-104.56,0.50,6.26,6.49,5.96,6.24
5,6.76,6.74,7.49,6.76,6.02,34.33,-0.31,-0.28,31.64,18.20,18.78,-92.67,0.51,6.24,6.47,5.95,6.22
6,6.73,6.69,7.50,6.73,5.95,33.81,-0.32,-0.29,32.42,17.82,18.13,-82.97,0.51,6.21,6.44,5.92,6.19
7,6.69,6.64,7.51,6.69,5.88,33.00,-0.33,-0.29,33.24,17.18,17.73,-79.35,0.51,6.17,6.40,5.88,6.14
8,6.66,6.58,7.52,6.66,5.81,32.09,-0.33,-0.30,34.11,16.45,17.15,-79.57,0.51,6.12,6.34,5.82,6.08
9,6.63,6.53,7.53,6.63,5.74,31.30,-0.33,-0.31,35.03,15.83,16.49,-83.31,0.51,6.07,6.29,5.78,6.04


In [660]:
result.to_csv('auto_ml_feature_predictor/result.csv', index=False)

In [661]:
validation_last_10 = complete_dataset[columns_to_model].tail(10)
ultimos_registros_last_10 = result

validation_last_10 = validation_last_10[columns_to_model].reset_index(drop=True)
ultimos_registros_last_10 = ultimos_registros_last_10[columns_to_model].reset_index(drop=True)

diferencia = validation_last_10 - ultimos_registros_last_10

display(diferencia)
# display(ultimos_registros_last_10)
# display(validation_last_10)

,Open,High,Low,SMA_20,EMA_20,Upper_Band,Middle_Band,Lower_Band,RSI,MACD,Signal,ADX,SlowK,SlowD,CCI,ATR
0,0.06,-0.08,-0.16,-0.04,0.08,0.28,-0.04,-0.36,-3.33,-0.26,-0.24,8.51,-27.50,-27.64,-21.02,0.04
1,-0.01,0.28,0.03,-0.08,0.12,-0.13,-0.08,-0.03,7.10,-0.19,-0.23,7.80,-6.07,-22.17,59.22,0.05
2,0.55,0.84,0.67,-0.02,0.21,-0.07,-0.02,0.03,14.03,-0.10,-0.20,5.67,29.94,-1.21,175.34,0.06
3,0.98,0.87,1.03,0.07,0.28,-0.03,0.07,0.17,14.09,-0.02,-0.16,3.62,51.45,25.11,179.57,0.05
4,0.99,0.79,1.14,0.12,0.34,-0.02,0.12,0.26,12.42,0.03,-0.13,1.70,57.67,46.35,161.68,0.02
5,0.88,0.93,1.07,0.18,0.41,0.02,0.18,0.35,15.67,0.08,-0.08,-0.47,58.46,55.86,163.70,0.01
6,1.09,1.03,1.14,0.23,0.46,0.02,0.23,0.44,13.29,0.11,-0.05,-2.74,56.80,57.64,154.83,-0.00
7,0.94,0.90,1.10,0.29,0.50,0.01,0.29,0.56,12.11,0.13,-0.01,-4.75,55.82,57.03,124.87,-0.02
8,0.87,0.85,1.07,0.33,0.53,0.00,0.33,0.65,12.85,0.14,0.02,-6.52,51.22,54.61,108.22,-0.03
9,0.91,0.80,1.00,0.37,0.58,-0.01,0.37,0.76,15.02,0.16,0.05,-8.00,51.36,52.80,95.77,-0.04
